In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [3]:
from lib.connected_db import PgsqlExecutor

In [4]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [5]:
db = PgsqlExecutor(snpsource)

In [138]:
SQL_GET_COMPANY_ID = """

select c.companyid from
ciqcompany c
             JOIN ciqsecurity s ON c.companyid = s.companyid
             JOIN ciqtradingitem ti ON ti.securityid = s.securityid
			 where ti.tradingitemid = %s
"""
rv = db.fetch_one_row(SQL_GET_COMPANY_ID, args=[416056350])
print (rv.query_data.companyid)

60839208


In [139]:

SQL_GET_FINANCIAL_DATA = """

   SELECT tradingitemid,
          dataitemvalue,
          dataitemid,
          financialperiodid,
          companyid,
          unittypevalue,
          latestperiodflag,
          fiscalyear
 from v_napshot_dcf_financial where companyid = 60839208  
 and fiscalyear >= {year} - 5 and fiscalyear < {year}
 and dataitemid = 4423

""".format(year=2020)

In [140]:
UNLEVERAGE_FREE_CASH_FLOW = 4423

In [141]:
def shifted_by(pd_series, row_shift):
    """
    get close price the day before shifted_by(data['close'], 1)
    :return: shifted_vals
    """
    shifted_vals = pd_series.iloc[:-row_shift]
    shifted_vals.index = pd_series.index[row_shift:]
    return shifted_vals

In [142]:
data = db.get_dataframe(SQL_GET_FINANCIAL_DATA)
data.head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid  companyid  \
0       67061158    3361.341625        4423         2017884777   60839208   
1       67061158   -1053.726750        4423         1947645590   60839208   
2       67061158    1517.769625        4423         1876474170   60839208   
3       67061158    -483.847875        4423         1829653489   60839208   
4       67061158       0.229500        4423         1777871175   60839208   

   unittypevalue  latestperiodflag  fiscalyear  
0        1000000                 1        2019  
1        1000000                 0        2018  
2        1000000                 0        2017  
3        1000000                 0        2016  
4        1000000                 0        2015

In [143]:
financial_data = data.sort_values(by=['fiscalyear'])
financial_data.head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid  companyid  \
69      412173566         0.2295        4423         1777871175   60839208   
34      266134120         0.2295        4423         1777871175   60839208   
39      290824399         0.2295        4423         1777871175   60839208   
44      291276745         0.2295        4423         1777871175   60839208   
49      311034402         0.2295        4423         1777871175   60839208   

    unittypevalue  latestperiodflag  fiscalyear  
69        1000000                 0        2015  
34        1000000                 0        2015  
39        1000000                 0        2015  
44        1000000                 0        2015  
49        1000000                 0        2015

In [144]:
financial_data = financial_data.groupby(
                ["fiscalyear"]).mean()

In [145]:
financial_data.head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid  \
fiscalyear                                                                
2015          362707968.5       0.229500      4423.0       1.777871e+09   
2016          362707968.5    -483.847875      4423.0       1.829653e+09   
2017          362707968.5    1517.769625      4423.0       1.876474e+09   
2018          362707968.5   -1053.726750      4423.0       1.947646e+09   
2019          362707968.5    3361.341625      4423.0       2.017885e+09   

             companyid  unittypevalue  latestperiodflag  
fiscalyear                                               
2015        60839208.0      1000000.0               0.0  
2016        60839208.0      1000000.0               0.0  
2017        60839208.0      1000000.0               0.0  
2018        60839208.0      1000000.0               0.0  
2019        60839208.0      1000000.0               1.0

In [146]:
financial_data.loc[:, 'last_year_value'] = shifted_by(financial_data['dataitemvalue'], 1)
financial_data.head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid  \
fiscalyear                                                                
2015          362707968.5       0.229500      4423.0       1.777871e+09   
2016          362707968.5    -483.847875      4423.0       1.829653e+09   
2017          362707968.5    1517.769625      4423.0       1.876474e+09   
2018          362707968.5   -1053.726750      4423.0       1.947646e+09   
2019          362707968.5    3361.341625      4423.0       2.017885e+09   

             companyid  unittypevalue  latestperiodflag  last_year_value  
fiscalyear                                                                
2015        60839208.0      1000000.0               0.0              NaN  
2016        60839208.0      1000000.0               0.0         0.229500  
2017        60839208.0      1000000.0               0.0      -483.847875  
2018        60839208.0      1000000.0               0.0      1517.769625  
2019        60839208.0      1000000.0               1.0     -1053.726750

In [147]:
from lib.linear_lib import LinearData

In [148]:
financial_data.loc[:, 'changes'] = (financial_data['dataitemvalue'] - financial_data['last_year_value']
                                   ) / abs(financial_data['last_year_value'])


In [149]:
new_data = financial_data.iloc[1:, :]

In [150]:
new_data.sort_values(by=['fiscalyear']).head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid  \
fiscalyear                                                                
2016          362707968.5    -483.847875      4423.0       1.829653e+09   
2017          362707968.5    1517.769625      4423.0       1.876474e+09   
2018          362707968.5   -1053.726750      4423.0       1.947646e+09   
2019          362707968.5    3361.341625      4423.0       2.017885e+09   

             companyid  unittypevalue  latestperiodflag  last_year_value  \
fiscalyear                                                                 
2016        60839208.0      1000000.0               0.0         0.229500   
2017        60839208.0      1000000.0               0.0      -483.847875   
2018        60839208.0      1000000.0               0.0      1517.769625   
2019        60839208.0      1000000.0               1.0     -1053.726750   

                changes  
fiscalyear               
2016       -2109.269608  
2017           4.136874  
2018          -1.694260  
2019           4.189956

In [151]:
for row in financial_data.iloc[1:, :].iterrows():
    print(row[0], row[1]["changes"])

2016 -2109.2696078431377
2017 4.136873598959179
2018 -1.6942600066857976
2019 4.189955673992331


In [152]:
new_data["changes"].mean()

-525.6592596442181

In [161]:
new_data["changes"].median() * 0.6

0.7327840776820144

In [91]:
type(new_data["changes"])

pandas.core.series.Series

In [28]:
import numpy as np

In [154]:
x = []
y = []

for row in new_data.iterrows():
    x.append([row[0]])
    y.append([row[1]["changes"]])

In [155]:
linear = LinearData(datax=np.array(x), 
                    datay=np.array(y))
linear.fit

LinearRegression()

In [156]:
linear.predict([[2018]])

-208.93188179680146

In [157]:
linear.predict(2020)

1057.9776295924094

In [158]:
linear.msr

334365.13799506077

In [160]:
linear.variance_score

0.6000160069600666

In [116]:
linear.variance_score * new_data["changes"].mean()

-0.006252734455846723

In [119]:
import math
math.log(linear.variance_score, 2)

-7.888729618674806

In [122]:
math.log(0.01, 2)

-6.643856189774724

-174467.0398768204

In [ ]:
if variance > 0.7:
    growth_rate = predict(next_year)
if new_data["changes"].mean() < 0:
    keep it
else:
    
    

In [115]:
adjusted_growth = 0.6 * -0.11
adjusted_growth

-0.066

In [66]:
-0.11 * (1 - 0.41)

-0.18644067796610167

In [59]:
linear.variance_score * 0.11

0.013093664921489776

In [ ]:
consistency = 0.7

In [60]:
example = LinearData(
datax=[[2013], [2014], [2015], [2016]],
datay=[[0.02], [0.01], [0.05], [0.07]]
)
print (round(example.msr, 3), example.variance_score)

0.0 0.7934065934066185


In [61]:
example.predict(2017)

0.08500000000000085

In [37]:
example = LinearData(
datax=[[2013], [2014], [2015], [2016]],
datay=[[0.02], [0.04], [-0.01], [0.09]]
)
print (round(example.msr, 3), example.variance_score)

0.001 0.2415094339622399


In [38]:
import numpy
numpy.mean([[0.02], [0.04], [-0.01], [0.09]])

0.034999999999999996

In [39]:
0.03 * 0.2

0.006

In [36]:
example.predict(2017)

0.11999999999999744

In [28]:
def adjust_growth_rate():
    if variance < 0.7:
        return growth_rate * variance
    else:
        return predict(next_year)